In [1]:
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv("churn-data-3333.csv")

In [2]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()
num_df.head()

,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
0,128,415,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,107,415,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,137,415,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,84,408,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,75,415,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3


In [3]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)
cat_df=cat_df.drop(['phone number'], axis=1)



#BOX-COX
lemda=0.5
num_df=(num_df**lemda)
num_df=num_df-1
num_df[num_df < 0]=0
num_df=num_df.div(lemda)
#End BOX-COX
#Z-score
#num_df=(num_df-num_df.min())/(num_df.std(ddof=0)) ##Z-score

#Log
#num_df=round(np.log(num_df.add(1)),2)
#num_df=num_df.replace([np.inf, -np.inf], np.nan)
#End Log

num_df=num_df.fillna(conData)

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df
#result_df_op=result_df_op.drop(['churn'], axis=1)
#result_df_op=np.nan_to_num(result_df_op)

X=result_df_op
#y=result_df['churn'] 
y=churn

from sklearn.model_selection import train_test_split  
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0, random_state=42)
X_train=X
X_test=X
y_train=y
y_test=y

In [4]:
X.shape

(3333, 19)

In [6]:
X_train=X
y_train=y
#Univariate Selection
from sklearn.feature_selection import SelectKBest, chi2
select_feature = SelectKBest(chi2, k=15).fit(X_train, y_train)
selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)

,Feature,Scores
17,international_plan,203.244178
2,number_vmail_messages,196.644554
15,number_customer_service_calls,150.736535
3,total_day_minutes,77.940600
5,total_day_charge,36.515370
18,voice_mail_plan,25.156959
6,total_eve_minutes,14.259848
13,total_intl_calls,8.091901
8,total_eve_charge,5.088664
12,total_intl_minutes,2.906557


In [7]:
y_train=y
x_train_chi = select_feature.transform(X)
x_train_chi.shape

(3333, 15)

In [14]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X,y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

2620 230 262 221
pod:  0.4575569358178054
pof:  0.08070175438596491
AUC:  0.6884275907159202
accuracy:  0.8523852385238524


In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

classifier=GaussianNB();
#params = {
#          "priors" : "None",
#          "var_smoothing" : 1e-9
#}
#create an list of var_smoothing to cross validate
#steps = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

grid = GridSearchCV(estimator=classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy', n_jobs=-1)

grid.fit(x_train_chi, y_train)

 
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 15 folds for each of 100 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    8.1s


{'var_smoothing': 0.001873817422860383}
GaussianNB(priors=None, var_smoothing=0.001873817422860383)
0.8788878887888789


[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:    9.9s finished


26710 23728 21590 27972
pod:  0.5643840038739357
pof:  0.4704389547563345
AUC:  0.5469725245588006


In [13]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=0.001873817422860383)

classifier.fit(x_train_chi,y_train)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

2825 25 379 104
pod:  0.2153209109730849
pof:  0.008771929824561403
AUC:  0.6032744905742617
accuracy:  0.8787878787878788


In [1]:
 #accuracy=(tp+tn)/(tp+fn+fp+tn)
accuracy=(26836+27976)/(27976+22462+22726+26836)
print ('accuracy: ',accuracy)

accuracy:  0.54812


In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Specify parameters
c_values = list(np.arange(1, 10))


param_grid = [
    {'C': c_values, 'penalty': ['l1'], 'solver' : ['liblinear'], 'multi_class' : ['ovr']},    
    {'C': c_values, 'penalty': ['l2'], 'solver' : ['liblinear', 'newton-cg', 'lbfgs'], 'multi_class' : ['ovr']}
]
 

grid = GridSearchCV(LogisticRegression(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 555.6min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 3133.4min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 3152.2min finished


{'C': 8, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'liblinear'}
LogisticRegression(C=8, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)
0.57708


In [15]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=8, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2771 79 392 91
pod:  0.18840579710144928
pof:  0.027719298245614036
AUC:  0.5803432494279176
accuracy:  0.8586858685868587


In [16]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=8, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2774 76 391 92
pod:  0.19047619047619047
pof:  0.02666666666666667
AUC:  0.5819047619047619
accuracy:  0.8598859885988599


In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  

# Specify parameters
c_values = list(np.arange(1, 10))
param_grid ={
   'n_neighbors': [3,5,11,19],
   'weights': ['uniform', 'distance'],
   'metric': ['euclidean', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1, cv=3, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 88.2min finished


{'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'uniform'}
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=19, p=2,
                     weights='uniform')
0.54095


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

# Specify parameters

Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
gammas = [0.001, 0.01, 0.1, 1, 1e-3, 1e-4]
param_grid = {'C': Cs, 'gamma' : gammas}

tuned_parameters = [{'kernel': ['rbf', 'linear'], 
                     'gamma': [0.001, 0.01, 0.1, 1, 1e-3, 1e-4], 
                     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]} ]

grid = GridSearchCV(svm.SVC(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 42 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 410.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 1993.6min


In [17]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(metric='manhattan', weights='uniform', n_neighbors=19 )  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


2848 2 399 84
pod:  0.17391304347826086
pof:  0.0007017543859649122
AUC:  0.5866056445461479
accuracy:  0.8796879687968797


In [18]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


2813 37 387 96
pod:  0.19875776397515527
pof:  0.012982456140350877
AUC:  0.5928876539174022
accuracy:  0.8727872787278728


In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


#clf = RandomForestClassifier()

# Choose some parameter combinations to try
#parameters = {'n_estimators': [4, 6, 9], 
#              'max_features': ['log2', 'sqrt','auto'], 
#              'criterion': ['entropy', 'gini'],
#              'max_depth': [2, 3, 5, 10], 
#              'min_samples_split': [2, 3, 5],
#              'min_samples_leaf': [1,5,8]
#             }

#model_params = {
#    'n_estimators': [50, 150, 250],
#    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
#    'min_samples_split': [2, 4, 6]
#}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed:  3.5min finished


{'bootstrap': True, 'max_depth': 80, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 300}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=80, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.9516951695169517


In [29]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=80, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2829 21 136 347
pod:  0.7184265010351967
pof:  0.007368421052631579
AUC:  0.8555290399912826
accuracy:  0.9528952895289529


In [20]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(n_estimators=100,max_depth=2,random_state=0)
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2849 1 473 10
pod:  0.020703933747412008
pof:  0.0003508771929824561
AUC:  0.5101765282772147


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

grid = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=10, n_jobs=-1)

grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)


In [21]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2711 139 131 352
pod:  0.7287784679089027
pof:  0.048771929824561404
AUC:  0.8400032690421706
accuracy:  0.918991899189919


In [22]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2713 137 133 350
pod:  0.7246376811594203
pof:  0.04807017543859649
AUC:  0.8382837528604119


In [23]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2811 39 131 352
pod:  0.7287784679089027
pof:  0.01368421052631579
AUC:  0.8575471286912935
accuracy:  0.948994899489949


In [24]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

2809 41 128 355
pod:  0.7349896480331263
pof:  0.014385964912280702
AUC:  0.8603018415604229
accuracy:  0.9492949294929492


In [25]:
df_x_train_chi = pd.DataFrame(x_train_chi)
df_x_train_chi.head()

#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print('tn_list: ',tn_list)
print ('fp_list: ',fp_list)
print ('fn_list: ',fn_list)
print ('tp_list: ',tp_list)


tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 2399 samples, validate on 600 samples
Epoch 1/30
2399/2399 [==============================] - 0s 195us/step - loss: 0.4372 - acc: 0.8512 - val_loss: 0.4455 - val_acc: 0.8417
Epoch 2/30
2399/2399 [==============================] - 0s 53us/step - loss: 0.4151 - acc: 0.8587 - val_loss: 0.4712 - val_acc: 0.8417
Epoch 3/30
2399/2399 [==============================] - 0s 54us/step - loss: 0.4211 - acc: 0.8587 - val_loss: 0.4694 - val_acc: 0.8417
Epoch 4/30
2399/2399 [==============================] - 0s 55us/step - loss: 0.4217 - acc: 0.8587 - val_loss: 0.4471 - val_acc: 0.8417
y2_pred:  [[0.22620884]
 [0.18191695]
 [0.28376943]
 [0.1968593 ]
 [0.23015067]
 [0.24896082]
 [0.21046281]
 [0.2776983 ]
 [0.28059313]
 [0.13973495]
 [0.2278344 ]
 [0.21063894]
 [0.35562384]
 [0.27536252]
 [0.21740815]
 [0.29969406]
 [0.17261985]
 [0.2610104 ]
 [0.1822336 ]
 [0.20870665]
 [

2399/2399 [==============================] - 0s 55us/step - loss: 0.4067 - acc: 0.8587 - val_loss: 0.4380 - val_acc: 0.8417
Epoch 4/30
2399/2399 [==============================] - 0s 55us/step - loss: 0.4067 - acc: 0.8587 - val_loss: 0.4354 - val_acc: 0.8417
Epoch 5/30
2399/2399 [==============================] - 0s 58us/step - loss: 0.4011 - acc: 0.8587 - val_loss: 0.4308 - val_acc: 0.8417
Epoch 6/30
2399/2399 [==============================] - 0s 57us/step - loss: 0.3969 - acc: 0.8587 - val_loss: 0.4633 - val_acc: 0.8417
Epoch 7/30
2399/2399 [==============================] - 0s 51us/step - loss: 0.4028 - acc: 0.8587 - val_loss: 0.4294 - val_acc: 0.8417
Epoch 8/30
2399/2399 [==============================] - 0s 51us/step - loss: 0.3962 - acc: 0.8587 - val_loss: 0.4287 - val_acc: 0.8417
Epoch 9/30
2399/2399 [==============================] - 0s 54us/step - loss: 0.3937 - acc: 0.8587 - val_loss: 0.4232 - val_acc: 0.8417
Epoch 10/30
2399/2399 [==============================] - 0s 54us/s

2399/2399 [==============================] - 1s 248us/step - loss: 0.4631 - acc: 0.8495 - val_loss: 0.4496 - val_acc: 0.8417
Epoch 2/30
2399/2399 [==============================] - 0s 53us/step - loss: 0.4290 - acc: 0.8587 - val_loss: 0.4403 - val_acc: 0.8417
Epoch 3/30
2399/2399 [==============================] - 0s 51us/step - loss: 0.4158 - acc: 0.8587 - val_loss: 0.4400 - val_acc: 0.8417
Epoch 4/30
2399/2399 [==============================] - 0s 52us/step - loss: 0.4088 - acc: 0.8587 - val_loss: 0.4408 - val_acc: 0.8417
Epoch 5/30
2399/2399 [==============================] - 0s 52us/step - loss: 0.4133 - acc: 0.8587 - val_loss: 0.4273 - val_acc: 0.8417
Epoch 6/30
2399/2399 [==============================] - 0s 53us/step - loss: 0.4025 - acc: 0.8587 - val_loss: 0.4146 - val_acc: 0.8417
Epoch 7/30
2399/2399 [==============================] - 0s 53us/step - loss: 0.3958 - acc: 0.8579 - val_loss: 0.4325 - val_acc: 0.8417
Epoch 8/30
2399/2399 [==============================] - 0s 58us/s

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 269us/step - loss: 0.4594 - acc: 0.8579 - val_loss: 0.4402 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 63us/step - loss: 0.4194 - acc: 0.8583 - val_loss: 0.4635 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 68us/step - loss: 0.4086 - acc: 0.8583 - val_loss: 0.4228 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 70us/step - loss: 0.3994 - acc: 0.8583 - val_loss: 0.4047 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 60us/step - loss: 0.3886 - acc: 0.8583 - val_loss: 0.3980 - val_acc: 0.8417
Epoch 6/30
2400/2400 [==============================] - 0s 58us/step - loss: 0.3867 - acc: 0.8575 - val_loss: 0.3952 - val_acc: 0.8417
Epoch 7/30
2400/2400 [==============================] - 0s 54us/step - loss: 0.3834 - acc: 0.8608 - val_loss: 0.4113 - val_acc: 0.8417
Epoch 8

...

2400/2400 [==============================] - 0s 60us/step - loss: 0.4008 - acc: 0.8583 - val_loss: 0.4225 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 55us/step - loss: 0.3932 - acc: 0.8579 - val_loss: 0.4670 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 60us/step - loss: 0.3969 - acc: 0.8579 - val_loss: 0.4229 - val_acc: 0.8383
Epoch 6/30
2400/2400 [==============================] - 0s 67us/step - loss: 0.3923 - acc: 0.8600 - val_loss: 0.4345 - val_acc: 0.8417
y2_pred:  [[0.2699577 ]
 [0.05879933]
 [0.08800837]
 [0.10183203]
 [0.09274095]
 [0.03968421]
 [0.11514696]
 [0.16837418]
 [0.11243838]
 [0.20656532]
 [0.06386945]
 [0.06062004]
 [0.06583127]
 [0.05318931]
 [0.15587655]
 [0.05988216]
 [0.06493142]
 [0.09065706]
 [0.04971203]
 [0.05097908]
 [0.06718186]
 [0.03160146]
 [0.06257495]
 [0.07820255]
 [0.10299706]
 [0.12891543]
 [0.07365873]
 [0.0509803 ]
 [0.07320705]
 [0.10759723]
 [0.05179816]
 [0.09408766]
 [0.1022982 ]

Epoch 8/30
2400/2400 [==============================] - 0s 60us/step - loss: 0.3903 - acc: 0.8583 - val_loss: 0.4144 - val_acc: 0.8417
Epoch 9/30
2400/2400 [==============================] - 0s 58us/step - loss: 0.3886 - acc: 0.8583 - val_loss: 0.4135 - val_acc: 0.8417
Epoch 10/30
2400/2400 [==============================] - 0s 59us/step - loss: 0.3870 - acc: 0.8583 - val_loss: 0.4279 - val_acc: 0.8417
Epoch 11/30
2400/2400 [==============================] - 0s 58us/step - loss: 0.3838 - acc: 0.8579 - val_loss: 0.4053 - val_acc: 0.8417
Epoch 12/30
2400/2400 [==============================] - 0s 56us/step - loss: 0.3829 - acc: 0.8579 - val_loss: 0.4109 - val_acc: 0.8417
Epoch 13/30
2400/2400 [==============================] - 0s 62us/step - loss: 0.3822 - acc: 0.8583 - val_loss: 0.4009 - val_acc: 0.8417
Epoch 14/30
2400/2400 [==============================] - 0s 60us/step - loss: 0.3749 - acc: 0.8583 - val_loss: 0.4114 - val_acc: 0.8417
Epoch 15/30
2400/2400 [===========================

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 1s 355us/step - loss: 0.5221 - acc: 0.8408 - val_loss: 0.4549 - val_acc: 0.8417
Epoch 2/30
2400/2400 [==============================] - 0s 58us/step - loss: 0.4131 - acc: 0.8583 - val_loss: 0.4434 - val_acc: 0.8417
Epoch 3/30
2400/2400 [==============================] - 0s 57us/step - loss: 0.4052 - acc: 0.8583 - val_loss: 0.4213 - val_acc: 0.8417
Epoch 4/30
2400/2400 [==============================] - 0s 58us/step - loss: 0.4062 - acc: 0.8583 - val_loss: 0.4231 - val_acc: 0.8417
Epoch 5/30
2400/2400 [==============================] - 0s 58us/step - loss: 0.4028 - acc: 0.8579 - val_loss: 0.4124 - val_acc: 0.8417
Epoch 6/30
2400/2400 [==============================] - 0s 56us/step - loss: 0.3961 - acc: 0.8592 - val_loss: 0.4282 - val_acc: 0.8417
Epoch 7/30
2400/2400 [==============================] - 0s 65us/step - loss: 0.4062 - acc: 0.8579 - val_loss: 0.4322 - val_acc: 0.8417
Epoch 8

2400/2400 [==============================] - 0s 82us/step - loss: 0.3957 - acc: 0.8621 - val_loss: 0.4797 - val_acc: 0.8233
Epoch 4/30
2400/2400 [==============================] - 0s 83us/step - loss: 0.3988 - acc: 0.8621 - val_loss: 0.4592 - val_acc: 0.8233
Epoch 5/30
2400/2400 [==============================] - 0s 80us/step - loss: 0.3904 - acc: 0.8633 - val_loss: 0.4659 - val_acc: 0.8233
Epoch 6/30
2400/2400 [==============================] - 0s 68us/step - loss: 0.3889 - acc: 0.8621 - val_loss: 0.4429 - val_acc: 0.8217
Epoch 7/30
2400/2400 [==============================] - 0s 73us/step - loss: 0.3896 - acc: 0.8625 - val_loss: 0.4437 - val_acc: 0.8233
Epoch 8/30
2400/2400 [==============================] - 0s 71us/step - loss: 0.3822 - acc: 0.8646 - val_loss: 0.4670 - val_acc: 0.8233
Epoch 9/30
2400/2400 [==============================] - 0s 73us/step - loss: 0.3801 - acc: 0.8625 - val_loss: 0.4364 - val_acc: 0.8217
Epoch 10/30
2400/2400 [==============================] - 0s 78us/s

Epoch 2/30
2400/2400 [==============================] - 0s 62us/step - loss: 0.4103 - acc: 0.8625 - val_loss: 0.4580 - val_acc: 0.8233
Epoch 3/30
2400/2400 [==============================] - 0s 61us/step - loss: 0.4043 - acc: 0.8629 - val_loss: 0.4799 - val_acc: 0.8233
Epoch 4/30
2400/2400 [==============================] - 0s 60us/step - loss: 0.3977 - acc: 0.8629 - val_loss: 0.4525 - val_acc: 0.8233
Epoch 5/30
2400/2400 [==============================] - 0s 63us/step - loss: 0.3941 - acc: 0.8633 - val_loss: 0.4592 - val_acc: 0.8233
Epoch 6/30
2400/2400 [==============================] - 0s 66us/step - loss: 0.3984 - acc: 0.8625 - val_loss: 0.4556 - val_acc: 0.8233
Epoch 7/30
2400/2400 [==============================] - 0s 65us/step - loss: 0.3835 - acc: 0.8629 - val_loss: 0.4582 - val_acc: 0.8233
y2_pred:  [[0.05154076]
 [0.13198009]
 [0.04015043]
 [0.02710101]
 [0.04281691]
 [0.24785027]
 [0.13118508]
 [0.13587463]
 [0.13249439]
 [0.06251976]
 [0.09631184]
 [0.03943524]
 [0.08984181

In [26]:
#RNN
def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=5, batch_size=128, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 

Train on 2399 samples, validate on 600 samples
Epoch 1/5
2399/2399 [==============================] - 2s 911us/step - loss: 0.5269 - acc: 0.8320 - val_loss: 0.4348 - val_acc: 0.8417
Epoch 2/5
2399/2399 [==============================] - 0s 81us/step - loss: 0.4024 - acc: 0.8587 - val_loss: 0.4252 - val_acc: 0.8417
Epoch 3/5
2399/2399 [==============================] - 0s 80us/step - loss: 0.3949 - acc: 0.8587 - val_loss: 0.4164 - val_acc: 0.8417
Epoch 4/5
2399/2399 [==============================] - 0s 86us/step - loss: 0.3861 - acc: 0.8587 - val_loss: 0.4132 - val_acc: 0.8417
Epoch 5/5
2399/2399 [==============================] - 0s 84us/step - loss: 0.3789 - acc: 0.8587 - val_loss: 0.4027 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 2s 865us/step - loss: 0.5453 - acc: 0.8217 - val_loss: 0.4369 - val_acc: 0.8417
Epoch 2/5
2400/2400 [==============================] - 0s 87us/step - loss: 0.4049 - acc: 0.8583 - val_loss: 0.4336 - val_acc: 0.8417
Epoch 3/5
2400/2400 [==============================] - 0s 82us/step - loss: 0.3981 - acc: 0.8583 - val_loss: 0.4292 - val_acc: 0.8417
Epoch 4/5
2400/2400 [==============================] - 0s 81us/step - loss: 0.3882 - acc: 0.8583 - val_loss: 0.4130 - val_acc: 0.8417
Epoch 5/5
2400/2400 [==============================] - 0s 82us/step - loss: 0.3790 - acc: 0.8583 - val_loss: 0.4050 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 2s 945us/step - loss: 0.5402 - acc: 0.8458 - val_loss: 0.4309 - val_acc: 0.8417
Epoch 2/5
2400/2400 [==============================] - 0s 84us/step - loss: 0.4022 - acc: 0.8583 - val_loss: 0.4254 - val_acc: 0.8417
Epoch 3/5
2400/2400 [==============================] - 0s 88us/step - loss: 0.3967 - acc: 0.8583 - val_loss: 0.4188 - val_acc: 0.8417
Epoch 4/5
2400/2400 [==============================] - 0s 86us/step - loss: 0.3909 - acc: 0.8583 - val_loss: 0.4114 - val_acc: 0.8417
Epoch 5/5
2400/2400 [==============================] - 0s 85us/step - loss: 0.3859 - acc: 0.8583 - val_loss: 0.4050 - val_acc: 0.8417
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 3s 1ms/step - loss: 0.5054 - acc: 0.8292 - val_loss: 0.4870 - val_acc: 0.8233
Epoch 2/5
2400/2400 [==============================] - 0s 91us/step - loss: 0.3914 - acc: 0.8629 - val_loss: 0.4531 - val_acc: 0.8233
Epoch 3/5
2400/2400 [==============================] - 0s 88us/step - loss: 0.3831 - acc: 0.8629 - val_loss: 0.4639 - val_acc: 0.8233
Epoch 4/5
2400/2400 [==============================] - 0s 86us/step - loss: 0.3753 - acc: 0.8629 - val_loss: 0.4433 - val_acc: 0.8233
Epoch 5/5
2400/2400 [==============================] - 0s 86us/step - loss: 0.3706 - acc: 0.8629 - val_loss: 0.4506 - val_acc: 0.8233
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

# Specify parameters

Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
gammas = [0.001, 0.01, 0.1, 1, 1e-3, 1e-4]
param_grid = {'C': Cs, 'gamma' : gammas}

tuned_parameters = [{'kernel': ['rbf', 'linear']} ]

grid = GridSearchCV(svm.SVC(), tuned_parameters, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 2 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 